# Editor 

#### This is the editor interface for Christopher Speed '24's Senior Thesis!

This notebook allows you to edit together a stand-up comedy special using several different editing paradigms (as explained below). 

It provides both fully automated and manual editing methods, and facilitates exporting a completed video.

### Import core libraries 

In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
from processing.load import setup_editor, load_edit, load_annotations
from editor.editor import edit_random, edit_simple, edit_complex
from editor.video import assemble

### Initialize State

At times, Jupyter notebooks can be fragile, requiring that you restart the kernel and reinitialize any variables. Given that this project involves processing lengthy video and audio files for editing, face detection, and other tasks, reloading and re-processing videos any time a restart is required would be incredibly unwieldy and inefficient, wasting considerable amounts of time.

To address this, we do the following to make our application *stateful* to minimize the negative impact of notebook restarts and errors:
- Until specified, each of the Python cells in this introductory portion of the notebook should be run **in order, top-to-bottom**. This provides a correct sequence of initialization for our edit and annotation data. 
- Each of the edit functions (`edit_simple`, `edit_complex`, and `edit_random`) saves the resulting edit sequence object as a pickled file to disk by default, allowing you to simply reinitialize the edit by unpickling the file in the case that a restart is required or you prefer the previous version of an edit.
	- these functions also permit specifying a new filename for the saved edits, allowing you to save multiple distinct edit sequences  

### Set edit session names

In [36]:
title_for_edit = "demo"

annotation_title = "demo_annotations"
clip_src_directory = ""
previous_edit_filename = ""  # replace these later to load
previous_annotation_filename = "" # replace these later to load

In [37]:
edit, annotations = setup_editor(
    new_edit_title=title_for_edit,
    new_annotation_title=annotation_title,
	clips_src_dir=clip_src_directory,
 	previous_edit_filename=previous_edit_filename,
  	previous_annotation_filename=previous_annotation_filename
)
for ed in edit.edit_list: print(ed)
for ann in annotations: print(ann)

Loading Prior Annotations
Saving clip annotations to demo_annotations.pkl
Annotation Data Loaded
Previous Edit exists! Loading it from disk.
Loading previous edit from demo.pkl
Previous Edit Data Loaded
VideoSegment(filename=MetaClipData(filename='C:\\Users\\chris\\Desktop\\Senior Thesis\\Workspace\\Clip_Left_Synchronized.mp4', normalized_detection_areas=array([0., 0., 0., ..., 0., 0., 0.]), avg_normalized_area=0.7225848894286726, clip_height=1080, clip_width=1920), start=9, end=9)
VideoSegment(filename=MetaClipData(filename='C:\\Users\\chris\\Desktop\\Senior Thesis\\Workspace\\Clip_Right_Synchronized.mp4', normalized_detection_areas=array([0., 0., 0., ..., 0., 0., 0.]), avg_normalized_area=0.42229644248772835, clip_height=1080, clip_width=1920), start=9, end=19761)
MetaClipData(filename='C:\\Users\\chris\\Desktop\\Senior Thesis\\Workspace\\Clip_Left_Synchronized.mp4', normalized_detection_areas=array([0., 0., 0., ..., 0., 0., 0.]), avg_normalized_area=0.7225848894286726, clip_height=1

# - Later cells may now be run arbitrarily; ordering requirements are relaxed -

## Simple Editor

In [38]:
start_clip = ""

In [ ]:
current_edit = load_edit(title_for_edit)
annotations = load_annotations(annotation_title)
edit = edit_simple(
    current_edit=current_edit,
    clips_and_annotations=annotations,
    starting_clip=annotations[0],
    threshold_frames=5,
    cut_frequency_threshold_frames=480,
    edit_start_time=0,
    strictness_amt=0,
    should_save=True
)
# NOTE: these numbers are entirely arbitrary and should be replaced to fit your needs
print(len(edit.edit_list)) # see how many cuts the model recommends

In [ ]:
# view the result of the above edit
for decision in edit.edit_list:
    print(decision)

## Complex Editor

In [27]:
start_clip = ""

In [ ]:
current_edit = load_edit(title_for_edit)
annotations = load_annotations(annotation_title)
edit = edit_complex(
    current_edit=current_edit,
    clips_and_annotations=annotations,
    starting_clip=annotations[0],
    threshold_frames=5,
    cut_frequency_threshold_frames=0,
    edit_start_time=0,
    strictness_amt=0,
    hold_start_frames=300,
    hold_end_frames=200,
    should_save=True
)
# NOTE: these numbers are entirely arbitrary and should be replaced to fit your needs
print(len(edit.edit_list)) # see how many cuts the model recommends

In [ ]:
# view the result of the above edit
for decision in edit.edit_list:
    print(decision)

## Random Editor

In [ ]:
current_edit = load_edit(title_for_edit)
annotations = load_annotations(annotation_title)
edit = edit_random(
    current_edit=current_edit,
    clips_and_annotations=annotations,
    num_cuts=60,
    should_save=True
)

for e in edit.edit_list[:8]:
    print(e)

In [35]:
# view the result of the above edit
for decision in edit.edit_list:
    print(decision)

[]


#### Visualize a specific frame for detection

In [35]:
import cv2

def save_frame_with_face_detection(video_filename, frame_number, output_filename):
    # Load the video
    cap = cv2.VideoCapture(video_filename)

    # Set the frame number
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    # Read the frame
    ret, frame = cap.read()

    # Check if frame is successfully read
    if not ret:
        print("Error: Could not read frame")
        return

    # Load the pre-trained face cascade classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Save the frame with face detection borders drawn
    cv2.imwrite(output_filename, frame)

    # Release the video capture object
    cap.release()

# Example usage:
left_video_filename = ""
right_video_filename = ""
frame_number = 1000 # Specify the frame number 
left_output_filename = left_video_filename[:-4] + '_output_frame_with_detection.png'
right_output_filename = right_video_filename[:-4] + '_output_frame_with_detection.png'

save_frame_with_face_detection(left_video_filename, frame_number, left_output_filename)
save_frame_with_face_detection(right_video_filename, frame_number, right_output_filename)

# Assemble Edit into Output Video

In [40]:
output_filename = "full output.mp4"
# edit = load_edit(title_for_edit)
final = assemble(edit.edit_list, fps=30, number_of_clips=35, verbose=True, output_video_filename=output_filename)

Edits to apply:
VideoSegment(filename='C:\\Users\\chris\\Desktop\\Senior Thesis\\Workspace\\Clip_Left_Synchronized.mp4', start=0, end=3)
VideoSegment(filename='C:\\Users\\chris\\Desktop\\Senior Thesis\\Workspace\\Clip_Right_Synchronized.mp4', start=4, end=699)
VideoSegment(filename='C:\\Users\\chris\\Desktop\\Senior Thesis\\Workspace\\Clip_Left_Synchronized.mp4', start=700, end=1183)
VideoSegment(filename='C:\\Users\\chris\\Desktop\\Senior Thesis\\Workspace\\Clip_Right_Synchronized.mp4', start=1184, end=1753)
VideoSegment(filename='C:\\Users\\chris\\Desktop\\Senior Thesis\\Workspace\\Clip_Left_Synchronized.mp4', start=1754, end=2313)
VideoSegment(filename='C:\\Users\\chris\\Desktop\\Senior Thesis\\Workspace\\Clip_Right_Synchronized.mp4', start=2314, end=2852)
VideoSegment(filename='C:\\Users\\chris\\Desktop\\Senior Thesis\\Workspace\\Clip_Left_Synchronized.mp4', start=2853, end=3388)
VideoSegment(filename='C:\\Users\\chris\\Desktop\\Senior Thesis\\Workspace\\Clip_Right_Synchronized.mp4